# Load packages 

In [1]:
# Installs
# !pip install geopandas
# !pip install rasterstats
# !pip install pyarrow

# !pip install torch torchvision rasterio


In [1]:
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# from geofeather.pygeos import to_geofeather, from_geofeather

import pandas as pd
import geopandas as gpd

#import pygeos
from rasterstats import zonal_stats
from scipy.stats import spearmanr
import shapely

from shapely.geometry import mapping, shape
from shapely import wkb
from shapely.wkb import loads as from_wkb

import rasterio

from pathlib import Path

from rasterio.warp import reproject, Resampling
from rasterio.windows import Window


from rasterio.windows import from_bounds
from rasterio.transform import from_origin

import fiona
from pathlib import Path
import numpy as np
import rasterio
from rasterio.warp import reproject
from rasterio.enums import Resampling as ResamplingEnums
from rasterio.features import rasterize

# Load data

## CISI index

In [ ]:
# # Set paths
# cisi_index_path = r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\data\010_degree\CISI_europe.feather"
# # cisi_index_path = "Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\data\CISI_europe.feather"
# cisi_index_clip_path = 'data/CISI_exposure_Global.feather'

# cisi_index_path
# # Read data CISI index
# cisi_index = pd.read_feather(cisi_index_path)
# cisi_index_clip = pd.read_feather(cisi_index_path)
# cisi_index_clip.head(3)

,CISI,Subscore_energy,Subscore_transportation,Subscore_water,Subscore_waste,Subscore_telecommunication,Subscore_healthcare,Subscore_education,geometry
0,0.015926,0.000000,0.014159,0.000000,0.000106,0.00000,0.000603,0.001058,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
1,0.000216,0.000000,0.000216,0.000000,0.000000,0.00000,0.000000,0.000000,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...
2,0.039630,0.002892,0.025552,0.000002,0.000012,0.00131,0.004327,0.005535,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...


In [2]:
cisi_010_path = r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\data\010_degree\CISI_europe.feather"
cisi_025_path = r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\data\025_degree\CISI_europe.feather"

In [44]:
# # Generate gpkg files from original CISI paths
# cisi_010 = pd.read_feather(cisi_010_path)
# cisi_025 = pd.read_feather(cisi_025_path)

# # Convert WKB to Shapely geometries
# cisi_010["geometry"] = cisi_010["geometry_wkb"].apply(wkb.loads)
# cisi_025["geometry"] = cisi_010["geometry_wkb"].apply(wkb.loads)

# # Create a GeoDataFrame
# gdf_010 = gpd.GeoDataFrame(cisi_010, geometry="geometry", crs="EPSG:4326")
# gdf_025 = gpd.GeoDataFrame(cisi_025, geometry="geometry", crs="EPSG:4326")

# # Drop WKB column
# gdf_010.drop(columns="geometry_wkb", inplace=True)
# gdf_025.drop(columns="geometry_wkb", inplace=True)

# # Save to file — choose one:
# gdf_010.to_file("CISI_010.gpkg", driver="GPKG")     
# gdf_025.to_file("CISI_025.gpkg", driver="GPKG")   

# print("[✓] GeoDataFrame saved.")

# File clipping

In [3]:
def clip_raster_to_bounds(input_path, output_path, min_lon, max_lon, min_lat, max_lat):  
    with rasterio.open(input_path) as src:
        print(src.bounds)

        # Build window from bounds
        window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
        transform = src.window_transform(window)
        data = src.read(window=window)

        meta = src.meta.copy()
        meta.update({
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": transform
        })

        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(data)

    print(f"Clipped raster saved to: {output_path}")



In [1]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.transform import Affine

def clip_raster_to_bounds_tif(input_path, output_path, min_lon, max_lon, min_lat, max_lat):
    # Open the source GeoTIFF
    with rasterio.open(input_path) as src:
        # Calculate the window to read based on geographic bounds
        window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
        
        # Read the clipped data
        clipped_data = src.read(window=window)
        
        # Calculate the transform for the clipped data
        clipped_transform = src.window_transform(window)
        
        # Update metadata
        meta = src.meta.copy()
        meta.update({
            "height": clipped_data.shape[1],
            "width": clipped_data.shape[2],
            "transform": clipped_transform
        })
        
        # Write to output GeoTIFF
        with rasterio.open(output_path, "w", **meta) as dst:
            dst.write(clipped_data)


In [ ]:
# Clipping
clip_raster_to_bounds(
    input_path=f"RAW_data\RAW_010\CISI_europe.feather",
    output_path=f"READY_data\CISI_clipped_010.feather",
    min_lon=-25,     # -25 degrees
    max_lon=32,      # 32 degrees (most eastern Finnish border)
    min_lat=32.41,   # Keep this original
    max_lat=80.91    # Keep this original
)


In [ ]:
# Example usage
clip_raster_to_bounds_tif(
    input_path=r"",
    output_path=r"READY_data",
    min_lon=-25,     # -25 degrees
    max_lon=32,      # 32 degrees (eastern Finnish border)
    min_lat=32.41,   # southern bound
    max_lat=80.91    # northern bound
)

RasterioIOError: Attempt to create 0x0 dataset is illegal,sizes must be larger than zero.

In [ ]:
import rasterio
from pyproj import CRS

path = r""
with rasterio.open(path) as src:
    print("CRS:", src.crs)
    print("Bounds:", src.bounds)

CRS: EPSG:4326
Bounds: BoundingBox(left=-180.001249265, bottom=-72.00041617728999, right=179.99874929500004, top=83.99958319871001)


In [9]:
# pip install geopandas shapely pyarrow pandas
from shapely.geometry import box
import geopandas as gpd
import pandas as pd

def clip_feather_bbox(
    input_path: str,
    output_path: str,
    min_lon: float, max_lon: float,
    min_lat: float, max_lat: float,
    crs: str = "EPSG:4326",
    preserve_wkb: bool = True
):
    """
    Clip a Feather table (with a 'geometry' column) to a lon/lat bbox
    and write a Feather file. Keeps geometry as WKB bytes by default.
    """
    # Read as plain Feather, then build geometry
    df = pd.read_feather(input_path)
    if "geometry" not in df.columns:
        raise ValueError(f"'geometry' column not found in {input_path}")

    geo_col0 = df["geometry"].iloc[0]
    if isinstance(geo_col0, (bytes, bytearray)):
        geom = gpd.GeoSeries.from_wkb(df["geometry"], crs=crs)
    else:
        # Could be WKB hex or WKT
        try:
            geom = gpd.GeoSeries.from_wkb(df["geometry"], crs=crs)
        except Exception:
            geom = gpd.GeoSeries.from_wkt(df["geometry"], crs=crs)

    gdf = gpd.GeoDataFrame(df.drop(columns=["geometry"]), geometry=geom, crs=crs)

    # Build bbox and clip
    bbox = box(min_lon, min_lat, max_lon, max_lat)
    # quick prefilter
    gdf = gdf[gdf.geometry.intersects(bbox)]
    # exact clip to the bbox polygon (cuts polygons at edges)
    clipped = gpd.clip(gdf, gpd.GeoDataFrame(geometry=[bbox], crs=crs))

    # Write Feather (preserve WKB bytes if that’s what you had)
    if preserve_wkb and isinstance(geo_col0, (bytes, bytearray)):
        out = clipped.drop(columns="geometry").copy()
        out["geometry"] = clipped.geometry.to_wkb()  # bytes
        out.reset_index(drop=True).to_feather(output_path)
    else:
        # GeoPandas-style Feather with embedded geometry metadata
        clipped.reset_index(drop=True).to_feather(output_path)

# ---------- USE EXACTLY LIKE THIS ----------
# 0.10° file
clip_feather_bbox(
    input_path=r"RAW_data\RAW_010\CISI_europe.feather",
    output_path=r"READY_data\CISI_clipped_010.feather",
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91
)

# 0.25° file (if you have it)
clip_feather_bbox(
    input_path=r"RAW_data\RAW_025\CISI_europe.feather",
    output_path=r"READY_data\CISI_clipped_025.feather",
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91
)



## Landuse paths

Categories of landuse:\
0: No data (unknown) \
1: Cropland\
2: Forest\
3: Grassland\
4: Shrubland\
5: Urban (impervious surface / artificial)\
6: Water\
7: Barren (bare land / non-vegetated)

In [ ]:
landuse_2020_path = ""

print(landuse_2020_path)

In [ ]:
landuse_SSP1_2030_path = "data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2030.tif"
landuse_SSP2_2030_path = "data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2030.tif" # this one was 2050 instead of 2030, redo this set!
landuse_SSP3_2030_path = "data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2030.tif"
landuse_SSP4_2030_path = "data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2030.tif"
landuse_SSP5_2030_path = "data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2030.tif"

landuse_2030 = ["landuse_SSP1_2030_path","landuse_SSP2_2030_path","landuse_SSP3_2030_path","landuse_SSP4_2030_path","landuse_SSP5_2030_path"]

print(landuse_2030)

['landuse_SSP1_2030_path', 'landuse_SSP2_2030_path', 'landuse_SSP3_2030_path', 'landuse_SSP4_2030_path', 'landuse_SSP5_2030_path']


In [16]:
landuse_SSP1_2050_path = "data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2050.tif"
landuse_SSP2_2050_path = "data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2050.tif"
landuse_SSP3_2050_path = "data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2050.tif"
landuse_SSP4_2050_path = "data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2050.tif"
landuse_SSP5_2050_path = "data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2050.tif"

landuse_2050 = ["landuse_SSP1_2050_path","landuse_SSP2_2050_path","landuse_SSP3_2050_path","landuse_SSP4_2050_path","landuse_SSP5_2050_path"]

print(landuse_2050)

['landuse_SSP1_2050_path', 'landuse_SSP2_2050_path', 'landuse_SSP3_2050_path', 'landuse_SSP4_2050_path', 'landuse_SSP5_2050_path']


In [17]:
landuse_SSP1_2100_path = "data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2100.tif"
landuse_SSP2_2100_path = "data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2100.tif"
landuse_SSP3_2100_path = "data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2100.tif"
landuse_SSP4_2100_path = "data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2100.tif"
landuse_SSP5_2100_path = "data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2100.tif"

landuse_2100 = ["landuse_SSP1_2100_path","landuse_SSP2_2100_path","landuse_SSP3_2100_path","landuse_SSP4_2100_path","landuse_SSP5_2100_path"]

print(landuse_2100)

['landuse_SSP1_2100_path', 'landuse_SSP2_2100_path', 'landuse_SSP3_2100_path', 'landuse_SSP4_2100_path', 'landuse_SSP5_2100_path']


In [18]:
# Split landuse by landuse class 
def split_landuse_by_class(input_tif, output_dir):
    """
    Splits a land use raster into separate binary rasters for each land use class.

    Parameters:
        input_tif (str or Path): Path to the input .tif land use raster.
        output_dir (str or Path): Directory where the class-specific rasters will be saved.

    Output:
        One binary .tif file per land use class in the output directory.
    """
    input_tif = Path(input_tif)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    with rasterio.open(input_tif) as src:
        landuse_data = src.read(1)
        profile = src.profile

    classes = np.unique(landuse_data)
    print(f"[INFO] Land use classes found in {input_tif.name}: {classes}")

    profile.update(dtype=rasterio.uint8, count=1)

    for lu_class in classes:
        binary_map = (landuse_data == lu_class).astype(rasterio.uint8)
        output_path = output_dir / f"{input_tif.stem}_class_{lu_class}.tif"

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(binary_map, 1)

        print(f"Saved class {lu_class} to {output_path}")


In [19]:
# Split landuse classes for 2030

# split_landuse_by_class(
#     input_tif="data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2030.tif",
#     output_dir="data/landuse_data/SSP1_RCP26/split_classes"
# )

# split_landuse_by_class(
#     input_tif="data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2030.tif",
#     output_dir="data/landuse_data/SSP2_RCP45/split_classes"
# )

# split_landuse_by_class(
#     input_tif="data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2030.tif",
#     output_dir="data/landuse_data/SSP3_RCP70/split_classes"
# )

# split_landuse_by_class(
#     input_tif="data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2030.tif",
#     output_dir="data/landuse_data/SSP4_RCP60/split_classes"
# )

# split_landuse_by_class(
#     input_tif="data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2030.tif",
#     output_dir="data/landuse_data/SSP5_RCP85/split_classes"
# )    

In [20]:
# Split landuse classes for 2050

split_landuse_by_class(
    input_tif="data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2050.tif",
    output_dir="data/landuse_data/SSP1_RCP26/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2050.tif",
    output_dir="data/landuse_data/SSP2_RCP45/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2050.tif",
    output_dir="data/landuse_data/SSP3_RCP70/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2050.tif",
    output_dir="data/landuse_data/SSP4_RCP60/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2050.tif",
    output_dir="data/landuse_data/SSP5_RCP85/split_classes"
)    

[INFO] Land use classes found in global_SSP1_RCP26_2050.tif: [0 1 2 3 4 5 6 7]
Saved class 0 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_0.tif
Saved class 1 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_1.tif
Saved class 2 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_2.tif
Saved class 3 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_3.tif
Saved class 4 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_4.tif
Saved class 5 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_5.tif
Saved class 6 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_6.tif
Saved class 7 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2050_class_7.tif
[INFO] Land use classes found in global_SSP2_RCP45_2050.tif: [0 1 2 3 4 5 6 7]
Saved class 0 to data\landuse_data\SSP2_RCP45\split_classes\global_SSP2_RCP45_2050

In [21]:
# Split landuse classes for 2100

split_landuse_by_class(
    input_tif="data/landuse_data/SSP1_RCP26/global_SSP1_RCP26_2100.tif",
    output_dir="data/landuse_data/SSP1_RCP26/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP2_RCP45/global_SSP2_RCP45_2100.tif",
    output_dir="data/landuse_data/SSP2_RCP45/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP3_RCP70/global_SSP3_RCP70_2100.tif",
    output_dir="data/landuse_data/SSP3_RCP70/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP4_RCP60/global_SSP4_RCP60_2100.tif",
    output_dir="data/landuse_data/SSP4_RCP60/split_classes"
)

split_landuse_by_class(
    input_tif="data/landuse_data/SSP5_RCP85/global_SSP5_RCP85_2100.tif",
    output_dir="data/landuse_data/SSP5_RCP85/split_classes"
)    

[INFO] Land use classes found in global_SSP1_RCP26_2100.tif: [0 1 2 3 4 5 6 7]
Saved class 0 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_0.tif
Saved class 1 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_1.tif
Saved class 2 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_2.tif
Saved class 3 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_3.tif
Saved class 4 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_4.tif
Saved class 5 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_5.tif
Saved class 6 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_6.tif
Saved class 7 to data\landuse_data\SSP1_RCP26\split_classes\global_SSP1_RCP26_2100_class_7.tif
[INFO] Land use classes found in global_SSP2_RCP45_2100.tif: [0 1 2 3 4 5 6 7]
Saved class 0 to data\landuse_data\SSP2_RCP45\split_classes\global_SSP2_RCP45_2100

### Generate gpkg file from .feather

In [54]:
# # Load the feather file
# df = pd.read_feather("CISI_exposure_with_pop_2030_025.feather")

# # Convert WKB to Shapely geometries
# df["geometry"] = df["geometry_wkb"].apply(wkb.loads)

# # Create a GeoDataFrame
# gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# # Drop WKB column
# gdf.drop(columns="geometry_wkb", inplace=True)

# # Save to file — choose one:
# gdf.to_file("CISI_europe_025.gpkg", driver="GPKG")     # GeoPackage (recommended)
# # gdf.to_file("CISI_exposure_with_pop_2030_025.geojson", driver="GeoJSON")  # GeoJSON
# # gdf.to_file("CISI_exposure_with_pop_2030_025.shp")  # Shapefile (limited column names)

# print("[✓] GeoDataFrame saved.")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [12]:
# pip install rasterio
import math
import os
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling

def _best_block(size, preferred=256):
    """Block size <= size and multiple of 16. If size < 16, disable tiling."""
    if size < 16:
        return None
    if preferred <= size:
        return preferred
    return max(16, (size // 16) * 16)

def aggregate_population_to_degree_grid(
    input_path: str,
    output_path: str,
    min_lon: float = -25.0,
    max_lon: float = 32.0,
    min_lat: float = 32.41,
    max_lat: float = 80.91,
    dst_res_deg: float = 0.25,       # 0.25° or 0.10°
    compress: str = "LZW",
    nodata_out: float = 0.0
):
    """
    Clip + aggregate a WGS84 (EPSG:4326) population raster (counts per 30")
    to a degree grid at dst_res_deg using SUM aggregation.
    """
    if min_lon >= max_lon or min_lat >= max_lat:
        raise ValueError("Invalid bbox: min must be < max for both lon/lat.")
    if dst_res_deg <= 0:
        raise ValueError("dst_res_deg must be positive.")

    dst_crs = CRS.from_epsg(4326)

    # Snap bbox to the output grid
    def snap_bounds(xmin, xmax, ymin, ymax, res):
        xmin_s = math.floor(xmin / res) * res
        xmax_s = math.ceil(xmax / res) * res
        ymin_s = math.floor(ymin / res) * res
        ymax_s = math.ceil(ymax / res) * res
        return xmin_s, xmax_s, ymin_s, ymax_s

    min_lon_s, max_lon_s, min_lat_s, max_lat_s = snap_bounds(min_lon, max_lon, min_lat, max_lat, dst_res_deg)

    width  = int(round((max_lon_s - min_lon_s) / dst_res_deg))
    height = int(round((max_lat_s - min_lat_s) / dst_res_deg))
    if width == 0 or height == 0:
        raise ValueError("Output size would be 0x0 — check bbox and resolution.")

    dst_transform = from_origin(min_lon_s, max_lat_s, dst_res_deg, dst_res_deg)
    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)

    with rasterio.open(input_path) as src:
        if src.crs is None or str(src.crs) not in ("EPSG:4326", "WGS84"):
            raise ValueError(f"Source CRS must be EPSG:4326 / WGS84; got {src.crs}")

        # use float64 to avoid overflow when summing many cells
        out_dtype = "float64"

        bx = _best_block(width, 256)
        by = _best_block(height, 256)
        use_tiling = (bx is not None and by is not None)

        dst_meta = src.meta.copy()
        dst_meta.update({
            "driver": "GTiff",
            "crs": dst_crs,
            "transform": dst_transform,
            "width": width,
            "height": height,
            "count": src.count,
            "dtype": out_dtype,
            "nodata": nodata_out,
            "compress": compress,
        })
        if use_tiling:
            dst_meta.update({"tiled": True, "blockxsize": bx, "blockysize": by})
        else:
            dst_meta.pop("tiled", None)
            dst_meta.pop("blockxsize", None)
            dst_meta.pop("blockysize", None)

        with rasterio.open(output_path, "w", **dst_meta) as dst:
            for b in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, b),
                    destination=rasterio.band(dst, b),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.sum,   # <-- aggregate counts
                    src_nodata=src.nodata,
                    dst_nodata=nodata_out,
                    init_dest_nodata=True,
                )

    print(f"✅ Wrote aggregated population: {output_path}")
    if use_tiling:
        print(f"    Tiled: {bx}×{by}")
    else:
        print("    Striped (tiling disabled for small dimension).")
    print(f"    EPSG:4326, res={dst_res_deg}°, size={height}×{width}px")
    print(f"    Bounds (snapped): lon[{min_lon_s}, {max_lon_s}], lat[{min_lat_s}, {max_lat_s}]")


In [14]:
# 0.25°
aggregate_population_to_degree_grid(
    input_path=r"data\Pop_ppp_2020_1km_Aggregated.tif",
    output_path=r"READY_data\pop_europe_025deg.tif",
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91,
    dst_res_deg=0.25
)

# 0.10°
aggregate_population_to_degree_grid(
    input_path=r"data\Pop_ppp_2020_1km_Aggregated.tif",
    output_path=r"READY_data\pop_europe_010deg.tif",
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91,
    dst_res_deg=0.10
)

✅ Wrote aggregated population: READY_data\pop_europe_025deg.tif
    Tiled: 224×192
    EPSG:4326, res=0.25°, size=195×228px
    Bounds (snapped): lon[-25.0, 32.0], lat[32.25, 81.0]
✅ Wrote aggregated population: READY_data\pop_europe_010deg.tif
    Tiled: 256×256
    EPSG:4326, res=0.1°, size=486×570px
    Bounds (snapped): lon[-25.0, 32.0], lat[32.4, 81.0]


In [8]:
import math
import os
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling

def _best_block(size, preferred=256):
    """Return a block size that is <= size and a multiple of 16.
    If size < 16, return None to signal 'disable tiling'."""
    if size < 16:
        return None
    # Use preferred (256) if it fits; else drop to the nearest multiple of 16
    if preferred <= size:
        return preferred
    # nearest multiple of 16 <= size
    return (size // 16) * 16

def reproject_clip_to_epsg4326(
    input_path: str,
    output_path: str,
    min_lon: float = -25.0,
    max_lon: float = 32.0,
    min_lat: float = 32.41,
    max_lat: float = 80.91,
    dst_res: float = 0.10,
    resampling: Resampling = Resampling.bilinear,
    compress: str = "LZW",
    nodata: float | None = None
):
    if min_lon >= max_lon or min_lat >= max_lat:
        raise ValueError("Invalid bbox: min must be < max for both lon/lat.")
    if dst_res <= 0:
        raise ValueError("dst_res must be positive (degrees per pixel).")

    dst_crs = CRS.from_epsg(4326)

    # snap to grid
    def snap_bounds(xmin, xmax, ymin, ymax, res):
        xmin_s = math.floor(xmin / res) * res
        xmax_s = math.ceil(xmax / res) * res
        ymin_s = math.floor(ymin / res) * res
        ymax_s = math.ceil(ymax / res) * res
        return xmin_s, xmax_s, ymin_s, ymax_s

    min_lon_s, max_lon_s, min_lat_s, max_lat_s = snap_bounds(min_lon, max_lon, min_lat, max_lat, dst_res)

    width  = int(round((max_lon_s - min_lon_s) / dst_res))
    height = int(round((max_lat_s - min_lat_s) / dst_res))
    if width == 0 or height == 0:
        raise ValueError("Output size would be 0x0 — check bbox and resolution.")

    dst_transform = from_origin(min_lon_s, max_lat_s, dst_res, dst_res)

    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)

    with rasterio.open(input_path) as src:
        if nodata is None:
            nodata = src.nodata

        # choose valid tile sizes (multiples of 16) or disable tiling
        bx = _best_block(width, preferred=256)
        by = _best_block(height, preferred=256)
        use_tiling = (bx is not None) and (by is not None)

        dst_meta = src.meta.copy()
        dst_meta.update({
            "driver": "GTiff",
            "crs": dst_crs,
            "transform": dst_transform,
            "width": width,
            "height": height,
            "nodata": nodata,
            "compress": compress,
        })
        if use_tiling:
            dst_meta.update({
                "tiled": True,
                "blockxsize": bx,
                "blockysize": by,
            })
        else:
            # fall back to strips (no blockx/ysize)
            dst_meta.pop("tiled", None)
            dst_meta.pop("blockxsize", None)
            dst_meta.pop("blockysize", None)

        with rasterio.open(output_path, "w", **dst_meta) as dst:
            for b in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, b),
                    destination=rasterio.band(dst, b),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=resampling,
                    dst_nodata=nodata,
                    src_nodata=nodata,
                    init_dest_nodata=True,
                )

    print(f"✅ Wrote reprojected+clipped raster: {output_path}")
    if use_tiling:
        print(f"    Tiled: {bx}×{by}")
    else:
        print("    Striped (tiling disabled for small dimension).")
    print(f"    EPSG:4326, res={dst_res}°, size={width}×{height}px")
    print(f"    Bounds (snapped): lon[{min_lon_s}, {max_lon_s}], lat[{min_lat_s}, {max_lat_s}]")


In [9]:
# 0.10
reproject_clip_to_epsg4326(
    input_path=r"data\GDP + Elec 2019\Real GDP\updated real GDP\2019\2019GDP.tif",   # Mollweide ESRI:54009
    output_path=r"READY_data\2019GDP_europe_010deg.tif",                            # EPSG:4326, 0.10°
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91,
    dst_res=0.10,                    # match CISI 0.10°
    resampling=Resampling.bilinear)   # continuous data -> bilinear is appropriate

✅ Wrote reprojected+clipped raster: READY_data\2019GDP_europe_010deg.tif
    Tiled: 256×256
    EPSG:4326, res=0.1°, size=570×486px
    Bounds (snapped): lon[-25.0, 32.0], lat[32.4, 81.0]


In [10]:
# 0.25
reproject_clip_to_epsg4326(
    input_path=r"data\GDP + Elec 2019\Real GDP\updated real GDP\2019\2019GDP.tif",   # Mollweide ESRI:54009
    output_path=r"READY_data\2019GDP_europe_025deg.tif",                            # EPSG:4326, 0.10°
    min_lon=-25, max_lon=32, min_lat=32.41, max_lat=80.91,
    dst_res=0.25,                    # match CISI 0.10°
    resampling=Resampling.bilinear)   # continuous data -> bilinear is appropriate

✅ Wrote reprojected+clipped raster: READY_data\2019GDP_europe_025deg.tif
    Tiled: 224×192
    EPSG:4326, res=0.25°, size=228×195px
    Bounds (snapped): lon[-25.0, 32.0], lat[32.25, 81.0]


# Clipping, resampling, aligning current data

In [15]:
# pip install rasterio
import math, os
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
import numpy as np

def make_template_raster(out_path, min_lon, max_lon, min_lat, max_lat, res_deg, crs="EPSG:4326", nodata=0.0):
    # snap bounds to grid
    def snap(v, step, fn): return fn(v / step) * step
    xmin = snap(min_lon, res_deg, math.floor)
    xmax = snap(max_lon, res_deg, math.ceil)
    ymin = snap(min_lat, res_deg, math.floor)
    ymax = snap(max_lat, res_deg, math.ceil)

    width  = int(round((xmax - xmin) / res_deg))
    height = int(round((ymax - ymin) / res_deg))
    if width <= 0 or height <= 0:
        raise ValueError("Template would be 0x0; check bounds/resolution.")

    transform = from_origin(xmin, ymax, res_deg, res_deg)
    profile = {
        "driver": "GTiff",
        "dtype": "float32",
        "count": 1,
        "width": width,
        "height": height,
        "crs": CRS.from_string(crs),
        "transform": transform,
        "nodata": nodata,
        "compress": "LZW",
        "tiled": True,
        "blockxsize": min(256, max(16, (width  // 16) * 16 or 16)),
        "blockysize": min(256, max(16, (height // 16) * 16 or 16)),
    }
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(np.full((1, height, width), nodata, dtype=np.float32))
    print(f"✅ Template written: {out_path}")
    print(f"   CRS={crs}, res={res_deg}°, size={height}×{width}, transform={transform}")

# Make your two templates (bounds you gave earlier)
make_template_raster("READY_data/CISI_template_0p25deg.tif", -25, 32, 32.41, 80.91, 0.25)
make_template_raster("READY_data/CISI_template_0p10deg.tif", -25, 32, 32.41, 80.91, 0.10)


✅ Template written: READY_data/CISI_template_0p25deg.tif
   CRS=EPSG:4326, res=0.25°, size=195×228, transform=| 0.25, 0.00,-25.00|
| 0.00,-0.25, 81.00|
| 0.00, 0.00, 1.00|
✅ Template written: READY_data/CISI_template_0p10deg.tif
   CRS=EPSG:4326, res=0.1°, size=486×570, transform=| 0.10, 0.00,-25.00|
| 0.00,-0.10, 81.00|
| 0.00, 0.00, 1.00|


In [16]:
import os, rasterio
from rasterio.warp import reproject, Resampling

def reproject_to_reference_grid(src_path, ref_path, out_path, agg="average", dst_nodata=0.0, compress="LZW"):
    resampling_map = {
        "sum": Resampling.sum, "average": Resampling.average,
        "nearest": Resampling.nearest, "bilinear": Resampling.bilinear, "mode": Resampling.mode
    }
    resampling = resampling_map[agg]
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)

    with rasterio.open(src_path) as src, rasterio.open(ref_path) as ref:
        profile = ref.profile.copy()
        # Use float64 when summing counts to prevent overflow
        profile.update({
            "dtype": "float64" if resampling is Resampling.sum else (src.dtypes[0] or "float32"),
            "count": src.count,
            "compress": compress,
            "nodata": dst_nodata
        })
        with rasterio.open(out_path, "w", **profile) as dst:
            for b in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, b),
                    destination=rasterio.band(dst, b),
                    src_transform=src.transform, src_crs=src.crs,
                    dst_transform=ref.transform, dst_crs=ref.crs,
                    resampling=resampling,
                    src_nodata=src.nodata, dst_nodata=dst_nodata,
                    init_dest_nodata=True,
                )
    print(f"✅ Aligned to reference: {out_path}")


In [18]:
# 0.25°
reproject_to_reference_grid(r"RAW_data\Pop_ppp_2020_1km_Aggregated.tif",
                            r"READY_data/CISI_template_025deg.tif",
                            r"READY_data/2020_pop_aligned_025deg.tif",
                            agg="sum")

reproject_to_reference_grid(r"data\GDP + Elec 2019\Real GDP\updated real GDP\2019\2019GDP.tif",
                            r"READY_data/CISI_template_025deg.tif",
                            r"READY_data/2019_gdp_aligned_025deg.tif",
                            agg="average")

reproject_to_reference_grid(r"data\corine\corine_land_class_2018\DATA\U2018_CLC2018_V2020_20u1.tif",
                            r"READY_data/CISI_template_025deg.tif",
                            r"READY_data/2018_landcover_aligned_025deg.tif",
                            agg="nearest")

✅ Aligned to reference: READY_data/2020_pop_aligned_025deg.tif
✅ Aligned to reference: READY_data/2019_gdp_aligned_025deg.tif
✅ Aligned to reference: READY_data/2018_landcover_aligned_025deg.tif


In [19]:
# 0.10°
reproject_to_reference_grid(r"RAW_data\Pop_ppp_2020_1km_Aggregated.tif",
                            r"READY_data/CISI_template_010deg.tif",
                            r"READY_data/2020_pop_aligned_010deg.tif",
                            agg="sum")

reproject_to_reference_grid(r"data\GDP + Elec 2019\Real GDP\updated real GDP\2019\2019GDP.tif",
                            r"READY_data/CISI_template_010deg.tif",
                            r"READY_data/2019_gdp_aligned_010deg.tif",
                            agg="average")

reproject_to_reference_grid(r"data\corine\corine_land_class_2018\DATA\U2018_CLC2018_V2020_20u1.tif",
                            r"READY_data/CISI_template_010deg.tif",
                            r"READY_data/2018_landcover_aligned_010deg.tif",
                            agg="nearest")


✅ Aligned to reference: READY_data/2020_pop_aligned_010deg.tif
✅ Aligned to reference: READY_data/2019_gdp_aligned_010deg.tif
✅ Aligned to reference: READY_data/2018_landcover_aligned_010deg.tif


# Align landuse raster to CISI gdf

In [25]:
# Check alignment of landuse raster vs CISI
def check_alignment(raster_path, gdf, resolution=0.25):
    bounds = gdf.total_bounds
    expected_transform = from_origin(bounds[0], bounds[3], resolution, resolution)


    with rasterio.open(raster_path) as src:
        aligned = (
            src.crs == gdf.crs and
            src.width == expected_width and
            src.height == expected_height and
            src.transform.almost_equals(expected_transform)
        )
    return aligned

# Align raster to grid 
def align_raster_to_gdf(src_raster_path, gdf, output_path, method='nearest', resolution=0.25):
    bounds = gdf.total_bounds
    width = int((bounds[2] - bounds[0]) / resolution)
    height = int((bounds[3] - bounds[1]) / resolution)
    transform = from_origin(bounds[0], bounds[3], resolution, resolution)

    with rasterio.open(src_raster_path) as src:
        src_data = src.read(1)
        dst_data = np.zeros((height, width), dtype=src_data.dtype)

        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=gdf.crs,
            resampling=Resampling.nearest if method == 'nearest' else Resampling.bilinear
        )

        profile = src.profile.copy()
        profile.update({
            "height": height,
            "width": width,
            "transform": transform,
            "crs": gdf.crs
        })

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(dst_data, 1)

    print(f"✅ Saved aligned raster to {output_path.name}")

## 2030

In [22]:
# Load the CISI grid
cisi_gdf_2030 = gpd.read_file("CISI_exposure_with_pop_2030_025.gpkg")

### SSP1

In [ ]:
# Parameters SSP1
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP1_RCP26/split_classes")  # folder with land use rasters for SSP1
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP1_RCP26_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP1_RCP26_2030_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP1_RCP26_2030_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP1_RCP26_2030_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP1_RCP26_2030_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP1_RCP26_2030_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP1_RCP26_2030_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP1_RCP26_2030_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP1_RCP26_2030_class_7_aligned.tif


In [ ]:
# Parameters SSP1
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP1_RCP26/split_classes")  # folder with land use rasters for SSP1
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP1_RCP26_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

[🔄] Aligning class 0...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_0_aligned.tif
[🔄] Aligning class 1...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_1_aligned.tif
[🔄] Aligning class 2...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_2_aligned.tif
[🔄] Aligning class 3...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_3_aligned.tif
[🔄] Aligning class 4...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_4_aligned.tif
[🔄] Aligning class 5...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_5_aligned.tif
[🔄] Aligning class 6...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_6_aligned.tif
[🔄] Aligning class 7...
[✓] Saved aligned raster to global_SSP1_RCP26_2030_class_7_aligned.tif


### SSP2

In [ ]:
# Parameters SSP2
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP2_RCP45/split_classes")  # folder with land use rasters for SSP2
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP2_RCP45_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

NameError: name 'check_alignment' is not defined

### SSP3

In [ ]:
# Parameters SSP3
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP3_RCP70/split_classes")  # folder with land use rasters for SSP3
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP3_RCP70_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

[🔄] Aligning class 0...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_0_aligned.tif
[🔄] Aligning class 1...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_1_aligned.tif
[🔄] Aligning class 2...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_2_aligned.tif
[🔄] Aligning class 3...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_3_aligned.tif
[🔄] Aligning class 4...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_4_aligned.tif
[🔄] Aligning class 5...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_5_aligned.tif
[🔄] Aligning class 6...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_6_aligned.tif
[🔄] Aligning class 7...
[✓] Saved aligned raster to global_SSP3_RCP70_2030_class_7_aligned.tif


### SSP4 

In [ ]:
# Parameters SSP4
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP4_RCP60/split_classes")  # folder with land use rasters for SSP4
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP4_RCP60_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

[🔄] Aligning class 0...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_0_aligned.tif
[🔄] Aligning class 1...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_1_aligned.tif
[🔄] Aligning class 2...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_2_aligned.tif
[🔄] Aligning class 3...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_3_aligned.tif
[🔄] Aligning class 4...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_4_aligned.tif
[🔄] Aligning class 5...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_5_aligned.tif
[🔄] Aligning class 6...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_6_aligned.tif
[🔄] Aligning class 7...
[✓] Saved aligned raster to global_SSP4_RCP60_2030_class_7_aligned.tif


### SSP5

In [ ]:
# Parameters SSP5
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP5_RCP85/split_classes")  # folder with land use rasters for SSP5
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP5_RCP85_2030_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

[🔄] Aligning class 0...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_0_aligned.tif
[🔄] Aligning class 1...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_1_aligned.tif
[🔄] Aligning class 2...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_2_aligned.tif
[🔄] Aligning class 3...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_3_aligned.tif
[🔄] Aligning class 4...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_4_aligned.tif
[🔄] Aligning class 5...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_5_aligned.tif
[🔄] Aligning class 6...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_6_aligned.tif
[🔄] Aligning class 7...
[✓] Saved aligned raster to global_SSP5_RCP85_2030_class_7_aligned.tif


## 2050

### SSP1

In [26]:
# Parameters SSP1
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP1_RCP26/split_classes")  # folder with land use rasters for SSP1
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP1_RCP26_2050_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP1_RCP26_2050_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP1_RCP26_2050_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP1_RCP26_2050_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP1_RCP26_2050_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP1_RCP26_2050_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP1_RCP26_2050_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP1_RCP26_2050_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP1_RCP26_2050_class_7_aligned.tif


### SSP2

In [27]:
# Parameters SSP2
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP2_RCP45/split_classes")  # folder with land use rasters for SSP2
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP2_RCP45_2050_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP2_RCP45_2050_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP2_RCP45_2050_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP2_RCP45_2050_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP2_RCP45_2050_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP2_RCP45_2050_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP2_RCP45_2050_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP2_RCP45_2050_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP2_RCP45_2050_class_7_aligned.tif


### SSP3

In [28]:
# Parameters SSP3
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP3_RCP70/split_classes")  # folder with land use rasters for SSP3
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP3_RCP70_2050_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP3_RCP70_2050_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP3_RCP70_2050_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP3_RCP70_2050_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP3_RCP70_2050_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP3_RCP70_2050_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP3_RCP70_2050_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP3_RCP70_2050_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP3_RCP70_2050_class_7_aligned.tif


### SSP4

In [29]:
# Parameters SSP4
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP4_RCP60/split_classes")  # folder with land use rasters for SSP4
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP4_RCP60_2050_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP4_RCP60_2050_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP4_RCP60_2050_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP4_RCP60_2050_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP4_RCP60_2050_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP4_RCP60_2050_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP4_RCP60_2050_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP4_RCP60_2050_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP4_RCP60_2050_class_7_aligned.tif


### SSP5

In [30]:
# Parameters SSP5
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP5_RCP85/split_classes")  # folder with land use rasters for SSP5
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP5_RCP85_2050_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP5_RCP85_2050_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP5_RCP85_2050_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP5_RCP85_2050_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP5_RCP85_2050_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP5_RCP85_2050_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP5_RCP85_2050_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP5_RCP85_2050_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP5_RCP85_2050_class_7_aligned.tif


## 2100

### SSP1

In [31]:
# Parameters SSP1
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP1_RCP26/split_classes")  # folder with land use rasters for SSP1
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP1_RCP26_2100_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP1_RCP26_2100_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP1_RCP26_2100_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP1_RCP26_2100_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP1_RCP26_2100_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP1_RCP26_2100_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP1_RCP26_2100_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP1_RCP26_2100_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP1_RCP26_2100_class_7_aligned.tif


### SSP2

In [32]:
# Parameters SSP2
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP2_RCP45/split_classes")  # folder with land use rasters for SSP2
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP2_RCP45_2100_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP2_RCP45_2100_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP2_RCP45_2100_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP2_RCP45_2100_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP2_RCP45_2100_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP2_RCP45_2100_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP2_RCP45_2100_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP2_RCP45_2100_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP2_RCP45_2100_class_7_aligned.tif


### SSP3

In [33]:
# Parameters SSP3
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP3_RCP70/split_classes")  # folder with land use rasters for SSP3
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP3_RCP70_2100_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP3_RCP70_2100_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP3_RCP70_2100_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP3_RCP70_2100_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP3_RCP70_2100_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP3_RCP70_2100_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP3_RCP70_2100_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP3_RCP70_2100_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP3_RCP70_2100_class_7_aligned.tif


### SSP4

In [34]:
# Parameters SSP4
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP4_RCP60/split_classes")  # folder with land use rasters for SSP4
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP4_RCP60_2100_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP4_RCP60_2100_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP4_RCP60_2100_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP4_RCP60_2100_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP4_RCP60_2100_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP4_RCP60_2100_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP4_RCP60_2100_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP4_RCP60_2100_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP4_RCP60_2100_class_7_aligned.tif


### SSP5

In [35]:
# Parameters SSP5
resolution = 0.25  # CISI resolution
input_dir = Path("data/landuse_data/SSP5_RCP85/split_classes")  # folder with land use rasters for SSP5
output_dir = input_dir / "aligned"
output_dir.mkdir(exist_ok=True)


# Batch align all landuse class rasters of a SSP
for class_id in range(8):  # assuming 8 land use classes (0 to 7)
    src_path = input_dir / f"global_SSP5_RCP85_2100_class_{class_id}.tif"
    clean_stem = src_path.stem.replace("global_", "")
    out_path = output_dir / f"{clean_stem}_aligned.tif"

    if not src_path.exists():
        print(f"File not found: {src_path.name}")
        continue

    if not check_alignment(src_path, cisi_gdf_2030, resolution):
        print(f"Aligning class {class_id}...")
        align_raster_to_gdf(src_path, cisi_gdf_2030, out_path, method='nearest', resolution=resolution)
    else:
        print(f"Already aligned: {src_path.name}")

Aligning class 0...
✅ Saved aligned raster to SSP5_RCP85_2100_class_0_aligned.tif
Aligning class 1...
✅ Saved aligned raster to SSP5_RCP85_2100_class_1_aligned.tif
Aligning class 2...
✅ Saved aligned raster to SSP5_RCP85_2100_class_2_aligned.tif
Aligning class 3...
✅ Saved aligned raster to SSP5_RCP85_2100_class_3_aligned.tif
Aligning class 4...
✅ Saved aligned raster to SSP5_RCP85_2100_class_4_aligned.tif
Aligning class 5...
✅ Saved aligned raster to SSP5_RCP85_2100_class_5_aligned.tif
Aligning class 6...
✅ Saved aligned raster to SSP5_RCP85_2100_class_6_aligned.tif
Aligning class 7...
✅ Saved aligned raster to SSP5_RCP85_2100_class_7_aligned.tif


# Combine landuse .tif files into 40-band GeoTIFF

In [2]:
def save_all_landuse_stacked_geotiff(
    base_dir,
    ssp_names,
    year,
    output_path,
    n_classes=8
):
    """
    Combines 40 aligned land use class rasters into one 40-band GeoTIFF.

    Parameters:
        base_dir (str or Path): Root folder containing SSP folders.
        ssp_names (list): List of SSP folder names.
        year (int): Year of the raster set.
        output_path (str or Path): Output .tif file.
        n_classes (int): Number of land use classes per SSP (class 1-7, 0 is empty)
    """
    base_dir = Path(base_dir)
    raster_paths = []

    for ssp in ssp_names:
        aligned_dir = base_dir / ssp / "split_classes" / "aligned"

        for class_id in range(n_classes):
            filename = f"{ssp}_{year}_class_{class_id}_aligned.tif"
            raster_path = aligned_dir / filename

            if not raster_path.exists():
                raise FileNotFoundError(f"Missing raster: {raster_path}")

            raster_paths.append(raster_path)

    # Use metadata from the first raster
    with rasterio.open(raster_paths[0]) as src:
        meta = src.meta.copy()
        meta.update(count=len(raster_paths))  # 40 bands expected

    # Write all rasters into a multi-band GeoTIFF
    with rasterio.open(output_path, 'w', **meta) as dst:
        for idx, path in enumerate(raster_paths, start=1):
            with rasterio.open(path) as src:
                dst.write(src.read(1), idx)

    print(f"Saved 40-band GeoTIFF: {output_path}")


In [4]:
# 2030
ssp_list_2030 = [
    "SSP1_RCP26",
    "SSP2_RCP45",
    "SSP3_RCP70",
    "SSP4_RCP60",
    "SSP5_RCP85"
]

save_all_landuse_stacked_geotiff(
    base_dir="data/landuse_data",
    ssp_names=ssp_list_2030,
    year=2030,
    output_path="landuse_stack_all_ssps_2030.tif"
)



Saved 40-band GeoTIFF: landuse_stack_all_ssps_2030.tif


# Align GDP raster with CISI to gdf

## Pre-rasterized GDP data (0.25 degrees) 2030 to 2100 with 5 year intervals

In [76]:
cisi_gdf_2030 = gpd.read_file("CISI_exposure_with_pop_2030_025.gpkg")

In [ ]:
# === PATHS ===
input_folder = Path(r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\GDP_gridded_025d")
output_folder = Path(r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\GDP_gridded_025d_clipped")
template_file = Path("data/025_degree/europe.tif")

output_folder.mkdir(parents=True, exist_ok=True)

# === LOAD TEMPLATE RASTER ===
with rasterio.open(template_file) as tmpl:
    tmpl_crs = tmpl.crs
    tmpl_transform = tmpl.transform
    tmpl_shape = (tmpl.height, tmpl.width)
    tmpl_meta = tmpl.meta.copy()

# === PROCESS EACH FILE ===
for tif_path in input_folder.glob("*.tif"):
    output_path = output_folder / f"aligned_{tif_path.name}"

    try:
        with rasterio.open(tif_path) as src:
            if src.crs != tmpl_crs:
                print(f"❌ Skipping {tif_path.name}: CRS mismatch")
                continue

            # Allocate array for aligned data
            dest_data = np.empty(tmpl_shape, dtype=np.float32)

            # Reproject and resample to template grid
            reproject(
                source=rasterio.band(src, 1),
                destination=dest_data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=tmpl_transform,
                dst_crs=tmpl_crs,
                resampling=ResamplingEnums.bilinear 
            )

            # Build output metadata
            out_meta = tmpl_meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": dest_data.shape[0],
                "width": dest_data.shape[1],
                "count": 1,
                "dtype": dest_data.dtype.name,
                "compress": "lzw"
            })

            # Write output raster
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(dest_data, 1)

            print(f"✅ Saved: {output_path.name}")

    except Exception as e:
        print(f"❌ Error processing {tif_path.name}: {e}")


❌ Error processing GDP2000.tif: aligned_GDP2000.tif: TIFFReadDirectory:Failed to read directory at offset 250
✅ Saved: aligned_GDP2001.tif
✅ Saved: aligned_GDP2002.tif
✅ Saved: aligned_GDP2003.tif
✅ Saved: aligned_GDP2004.tif
✅ Saved: aligned_GDP2005.tif
✅ Saved: aligned_GDP2006.tif
✅ Saved: aligned_GDP2007.tif
✅ Saved: aligned_GDP2008.tif
✅ Saved: aligned_GDP2009.tif
✅ Saved: aligned_GDP2010.tif
✅ Saved: aligned_GDP2011.tif
✅ Saved: aligned_GDP2012.tif
✅ Saved: aligned_GDP2013.tif
✅ Saved: aligned_GDP2014.tif
✅ Saved: aligned_GDP2015.tif
✅ Saved: aligned_GDP2016.tif
✅ Saved: aligned_GDP2017.tif
✅ Saved: aligned_GDP2018.tif
✅ Saved: aligned_GDP2019.tif
✅ Saved: aligned_GDP2020.tif
✅ Saved: aligned_GDP2025_ssp1.tif
✅ Saved: aligned_GDP2025_ssp2.tif
✅ Saved: aligned_GDP2025_ssp3.tif
✅ Saved: aligned_GDP2025_ssp4.tif
✅ Saved: aligned_GDP2025_ssp5.tif
✅ Saved: aligned_GDP2030_ssp1.tif
✅ Saved: aligned_GDP2030_ssp2.tif
✅ Saved: aligned_GDP2030_ssp3.tif
✅ Saved: aligned_GDP2030_ssp4.tif
✅ Sa

# Align population raster to CISI gdf

In [ ]:
# 2010
world_pop_2010_SSP2_merkens = "data/output/resample/clipped_pop_2010.tif"
world_pop_2010_SSP1 = "data/population_v2/SSP1_2010_EU_UK.tif"
world_pop_2010_SSP2 = "data/population_v2/SSP2_2010_EU_UK.tif"
world_pop_2010_SSP3 = "data/population_v2/SSP3_2010_EU_UK.tif"
world_pop_2010_SSP4 = "data/population_v2/SSP4_2010_EU_UK.tif"
world_pop_2010_SSP5 = "data/population_v2/SSP5_2010_EU_UK.tif"



# 2030
world_pop_2030_SSP1 = "data/population_v2/SSP1_2030_EU_UK.tif"
world_pop_2030_SSP2 = "data/population_v2/SSP2_2030_EU_UK.tif"
world_pop_2030_SSP3 = "data/population_v2/SSP3_2030_EU_UK.tif"
world_pop_2030_SSP4 = "data/population_v2/SSP4_2030_EU_UK.tif"
world_pop_2030_SSP5 = "data/population_v2/SSP5_2030_EU_UK.tif"
world_pop_2030 = [world_pop_2030_SSP1, world_pop_2030_SSP2, world_pop_2030_SSP3, world_pop_2030_SSP4, world_pop_2030_SSP5]

# 2050
world_pop_2050_SSP1 = "data/population_v2/SSP1_2050_EU_UK.tif"
world_pop_2050_SSP2 = "data/population_v2/SSP2_2050_EU_UK.tif"
world_pop_2050_SSP3 = "data/population_v2/SSP3_2050_EU_UK.tif"
world_pop_2050_SSP4 = "data/population_v2/SSP4_2050_EU_UK.tif"
world_pop_2050_SSP5 = "data/population_v2/SSP5_2050_EU_UK.tif"
world_pop_2050 = [world_pop_2050_SSP1, world_pop_2050_SSP2, world_pop_2050_SSP3, world_pop_2050_SSP4, world_pop_2050_SSP5]

# 2100
world_pop_2100_SSP1 = "data/population_v2/SSP1_2100_EU_UK.tif"
world_pop_2100_SSP2 = "data/population_v2/SSP2_2100_EU_UK.tif"
world_pop_2100_SSP3 = "data/population_v2/SSP3_2100_EU_UK.tif"
world_pop_2100_SSP4 = "data/population_v2/SSP4_2100_EU_UK.tif"
world_pop_2100_SSP5 = "data/population_v2/SSP5_2100_EU_UK.tif"
world_pop_2100 = [world_pop_2100_SSP1, world_pop_2100_SSP2, world_pop_2100_SSP3, world_pop_2100_SSP4, world_pop_2100_SSP5]



# Convert file to .tif

In [4]:
df = pd.read_feather("cisi_index_pop_all_years.feather")
df['geometry'] = df['geometry_wkb'].apply(shapely.wkb.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(df.columns)


Index(['CISI', 'Subscore_energy', 'Subscore_transportation', 'Subscore_water',
       'Subscore_waste', 'Subscore_telecommunication', 'Subscore_healthcare',
       'Subscore_education', 'total_pop_2050_SSP1', 'total_pop_2050_SSP2',
       'total_pop_2050_SSP3', 'total_pop_2050_SSP4', 'total_pop_2050_SSP5',
       'total_pop_2100_SSP1', 'total_pop_2100_SSP2', 'total_pop_2100_SSP3',
       'total_pop_2100_SSP4', 'total_pop_2100_SSP5', 'geometry_wkb',
       'geometry'],
      dtype='object')


In [6]:

def feather_to_raster(
    feather_path: str,
    reference_raster_path: str,
    value_column: str,
    output_tif_path: str,
    dtype: str = "float32"
):
    # Load .feather into GeoDataFrame
    df = pd.read_feather(feather_path)
    df['geometry'] = df['geometry_wkb'].apply(shapely.wkb.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

    # Match CRS to reference raster
    with rasterio.open(reference_raster_path) as ref:
        gdf = gdf.to_crs(ref.crs)
        transform = ref.transform
        crs = ref.crs
        out_shape = (ref.height, ref.width)

    # Prepare shapes (geometry, value)
    shapes = zip(gdf.geometry, gdf[value_column])

    # Rasterize
    raster = rasterize(
        shapes=shapes,
        out_shape=out_shape,
        transform=transform,
        fill=0,
        dtype=dtype
    )

    # Write to .tif
    with rasterio.open(
        output_tif_path,
        "w",
        driver="GTiff",
        height=out_shape[0],
        width=out_shape[1],
        count=1,
        dtype=raster.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(raster, 1)

    print(f"Saved raster to {output_tif_path}")

    

In [ ]:
feather_to_raster(
    feather_path="cisi_index_pop_all_years.feather",
    reference_raster_path="GDP_clipped_files_025d/GDP2030__025_ssp1_clipped.tif",
    value_column="CISI",
    output_tif_path="cisi_index_pop_all_years.tif"
)



Saved raster to cisi_index_pop_all_years.tif


In [1]:
# # === Step 2: Define output raster grid ===
# res = 0.25  # grid resolution in degrees
# minx, miny, maxx, maxy = gdf.total_bounds
# width = int((maxx - minx) / res)
# height = int((maxy - miny) / res)
# transform = from_origin(minx, maxy, res, res)

# # === Step 3: Identify all numeric columns to rasterize (exclude geometry only) ===
# value_cols = gdf.select_dtypes(include=[np.number]).columns
# print("Rasterizing columns:", value_cols.tolist())

# # === Step 4: Rasterize each numeric column into a separate band ===
# layers = []
# for col in value_cols:
#     print(f"Rasterizing: {col}")
#     raster = rasterize(
#         [(geom, val) for geom, val in zip(gdf.geometry, gdf[col])],
#         out_shape=(height, width),
#         transform=transform,
#         fill=0,
#         dtype="float32"
#     )
#     layers.append(raster)

# # === Step 5: Stack into 3D tensor: [bands, height, width] ===
# tensor = np.stack(layers, axis=0)

# # === Step 6: Normalize each band to [0, 1] (optional but CNN-friendly) ===
# eps = 1e-6
# tensor = (tensor - tensor.min(axis=(1,2), keepdims=True)) / (
#     tensor.max(axis=(1,2), keepdims=True) - tensor.min(axis=(1,2), keepdims=True) + eps
# )

# # === Step 7: Save as multi-band GeoTIFF ===
# with rasterio.open(
#     "CISI_exposure_with_pop_2030_multiband.tif",
#     "w",
#     driver="GTiff",
#     height=height,
#     width=width,
#     count=tensor.shape[0],  # number of bands
#     dtype="float32",
#     crs=gdf.crs,
#     transform=transform,
# ) as dst:
#     for i in range(tensor.shape[0]):
#         dst.write(tensor[i], i + 1)  # Bands are 1-indexed

# print("Multi-band GeoTIFF saved:", "CISI_exposure_with_pop_2030_multiband.tif")
# print("Shape of tensor:", tensor.shape)  # [bands, height, width]

# Generate label file (CISI 2020)

KeyError: 'year'

# Check column 

In [14]:
import rasterio

path = r"C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\025_degree_data\GDP_clipped_files_025d\GDP2030__025_ssp1_clipped.tif"

with rasterio.open(path) as src:
    print("File:", path)
    print("Width x Height:", src.width, "x", src.height)
    print("Number of bands:", src.count)
    print("CRS:", src.crs)
    print("Transform (affine):", src.transform)
    print("NoData value:", src.nodata)

    # band "names" are usually just 1..count, but we can list descriptions
    for i in range(1, src.count + 1):
        desc = src.descriptions[i - 1] if src.descriptions[i - 1] else f"Band {i}"
        print(f" Band {i}: {desc}, dtype={src.dtypes[i - 1]}")


File: C:\Users\Gebruiker\Desktop\Thesis\FutureCISI-main\FutureCISI-main\025_degree_data\GDP_clipped_files_025d\GDP2030__025_ssp1_clipped.tif
Width x Height: 228 x 194
Number of bands: 1
CRS: EPSG:4326
Transform (affine): | 0.25, 0.00,-25.00|
| 0.00,-0.25, 80.91|
| 0.00, 0.00, 1.00|
NoData value: nan
 Band 1: Band 1, dtype=float32


In [ ]:
df = pd.read_feather(025_degree_data\GDP_clipped_files_025d\GDP2030__025_ssp1_clipped.tif
df['geometry'] = df['geometry_wkb'].apply(shapely.wkb.loads))
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(df.columns)


SyntaxError: invalid decimal literal (2507891313.py, line 1)